In [ ]:
import tensorflow as tf 
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
tf.__version__

## Data Preprocessing

### Generating Images for Training Set

In [ ]:
trainDataGen = ImageDataGenerator(
    rescale = 1.0 / 255.0,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True 
)

trainingSet = trainDataGen.flow_from_directory(
    'data/training-set',
    target_size = (128, 128),
    batch_size = 32,
    class_mode = 'categorical'
)

In [ ]:
dog_classes_list = [0] * 26
dog_classes = trainingSet.class_indices
for i in dog_classes:
    dog_classes_list[dog_classes[i]] = i

dog_classes

## Writing classes in json file

In [ ]:
import json 

dog_classes_json = json.dumps(dog_classes)

with open('dog_classes.json', 'w+') as file:
    file.write(dog_classes_json)

### Generating images for Test set

In [ ]:
testDataGen = ImageDataGenerator(
    rescale = 1.0 / 255.0
)

testSet = testDataGen.flow_from_directory(
    'data/test-set',
    target_size = (128, 128),
    batch_size = 32,
    class_mode = 'categorical'
)

## Building The CNN

In [ ]:
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, InputLayer, BatchNormalization, Dropout, Activation
from keras.models import Sequential

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=( 128, 128, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
#model.add(Dropout(0.5))
model.add(Dense(26))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.fit(x = trainingSet, validation_data = testSet, epochs = 10)

In [ ]:
import numpy as np 
from keras.preprocessing import image

# Loads image in PIL format
test_image = image.load_img('./backup/data/single_prediction/4.jpg', target_size = (128, 128))
test_image = image.img_to_array(test_image) / 255.      
test_image = np.expand_dims(test_image, axis = 0)        
result = model.predict_proba(test_image)

print(result[0])

In [ ]:
maxIndex = -1
maxProb = 0
for i in range(len(result[0])):
    if result[0][i] > maxProb:
        maxIndex = i 
        maxProb = result[0][i]

print(maxProb)
print(maxIndex)
print(dog_classes_list[maxIndex])

In [ ]:
model.save('model')